In [3]:
# 1. Logging Setup (run first)

import logging

class CustomFormatter(logging.Formatter):
    """Custom formatter with colors for notebook output"""
    grey = "\x1b[38;21m"
    blue = "\x1b[38;5;39m"
    yellow = "\x1b[38;5;226m"
    red = "\x1b[38;5;196m"
    reset = "\x1b[0m"

    def __init__(self):
        super().__init__(fmt="%(asctime)s - %(levelname)s - %(message)s",
                         datefmt="%Y-%m-%d %H:%M:%S")
        self.FORMATS = {
            logging.DEBUG: self.grey,
            logging.INFO: self.blue,
            logging.WARNING: self.yellow,
            logging.ERROR: self.red
        }

    def format(self, record):
        color = self.FORMATS.get(record.levelno, self.grey)
        log_fmt = f"{color}%(asctime)s - %(levelname)s - %(message)s{self.reset}"
        formatter = logging.Formatter(log_fmt, datefmt="%Y-%m-%d %H:%M:%S")
        return formatter.format(record)

# Setup logger
logger = logging.getLogger("YOLOComparison")
logger.setLevel(logging.INFO)
if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(CustomFormatter())
    logger.addHandler(console_handler)
logger.info("Logger initialized")

2025-06-08 07:53:29 - INFO - Logger initialized
INFO:YOLOComparison:Logger initialized


In [4]:
# Install all required packages (run this first if you get ModuleNotFoundError)
!pip install ultralytics sahi pycocotools pandas seaborn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00


In [6]:
# 2. Imports & Config (run after logging setup)

import os
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import matplotlib.patches as patches
from ultralytics import YOLO, __version__ as yolo_version
from sklearn.metrics import confusion_matrix

# Google Drive mounting (run only in Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Path configs (Change if needed)
BASE_DIR = '/content/drive/MyDrive/new scope model'
DATA_YAML = f'{BASE_DIR}/data.yaml'
DATASET_DIR = '/content/drive/MyDrive/Samplesmall_dataset'

logger.info(f"Using Ultralytics YOLO version: {yolo_version}")
for path in [BASE_DIR, DATA_YAML, DATASET_DIR]:
    if not os.path.exists(path):
        logger.error(f"Path not found: {path}")

2025-06-08 07:58:02 - INFO - Using Ultralytics YOLO version: 8.3.152
INFO:YOLOComparison:Using Ultralytics YOLO version: 8.3.152


Mounted at /content/drive


In [7]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_comparison_grid(images, detections_list, class_names, method_names, save_path):
    """
    images: list of np.array images [img1, img2]
    detections_list: list of lists, shape [n_methods][n_images][detections]
    class_names: list of str
    method_names: list of str, e.g. ['YOLOv8', 'Enhanced YOLO', 'SAHI']
    save_path: where to save the output image
    """
    n_images = len(images)
    n_methods = len(method_names)
    fig, axes = plt.subplots(n_images, n_methods, figsize=(5 * n_methods, 5 * n_images))

    for i in range(n_images):
        for j in range(n_methods):
            ax = axes[i, j] if n_images > 1 else axes[j]
            ax.imshow(images[i])
            # Draw detections for this method/image
            for det in detections_list[j][i]:
                bbox = det['bbox']
                label = class_names[det['class_id']]
                conf = det['confidence']
                rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
                                         linewidth=2, edgecolor='r', facecolor='none')
                ax.add_patch(rect)
                ax.text(bbox[0], bbox[1]-5, f'{label}: {conf:.2f}',
                        color='white', bbox=dict(facecolor='red', alpha=0.5))
            if i == 0:
                ax.set_title(method_names[j])
            ax.axis('off')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] Saved comparison grid to {save_path}")

In [8]:
# 3. ResultsVisualizer Class (data visualization)

class ResultsVisualizer:
    """Handles all visualization tasks"""
    def __init__(self, base_dir):
        self.base_dir = base_dir
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.results_dir = os.path.join(base_dir, f'comparison_results_{self.timestamp}')
        self.dirs = {
            'plots': os.path.join(self.results_dir, 'plots'),
            'metrics': os.path.join(self.results_dir, 'metrics'),
            'detections': os.path.join(self.results_dir, 'detection_examples'),
            'logs': os.path.join(self.results_dir, 'logs')
        }
        for dir_path in self.dirs.values():
            os.makedirs(dir_path, exist_ok=True)
        logger.info(f"Created results directory at {self.results_dir}")

    def plot_metrics_comparison(self, metrics_dict):
        plt.figure(figsize=(12, 8))
        df = pd.DataFrame(metrics_dict).T
        ax = df.plot(kind='bar', width=0.8)
        plt.title('Performance Comparison Across Models')
        plt.xlabel('Model Type')
        plt.ylabel('Score')
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f')
        plt.tight_layout()
        save_path = os.path.join(self.dirs['plots'], 'metrics_comparison.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        csv_path = os.path.join(self.dirs['metrics'], 'metrics_comparison.csv')
        df.to_csv(csv_path)
        markdown_path = os.path.join(self.dirs['metrics'], 'metrics_summary.md')
        with open(markdown_path, 'w') as f:
            f.write("# Model Performance Comparison\n\n")
            f.write(df.to_markdown())
        return df

    def plot_confusion_matrix(self, true_labels, pred_labels, class_names, model_name):
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(true_labels, pred_labels)
        sns.heatmap(cm, annot=True, fmt='d',
                    xticklabels=class_names,
                    yticklabels=class_names)
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        save_path = os.path.join(self.dirs['plots'], f'confusion_matrix_{model_name}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_precision_recall_curves(self, precisions, recalls, model_names):
        plt.figure(figsize=(10, 8))
        for i, model_name in enumerate(model_names):
            plt.plot(recalls[i], precisions[i], label=model_name)
        plt.title('Precision-Recall Curves')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend()
        plt.grid(True)
        save_path = os.path.join(self.dirs['plots'], 'precision_recall_curves.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_per_class_map(self, class_maps, class_names, model_names):
        df = pd.DataFrame(class_maps, index=model_names, columns=class_names)
        plt.figure(figsize=(15, 8))
        ax = df.plot(kind='bar', width=0.8)
        plt.title('Per-Class mAP Comparison')
        plt.xlabel('Model')
        plt.ylabel('mAP')
        plt.legend(title='Classes', bbox_to_anchor=(1.05, 1), loc='upper left')
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f', rotation=90)
        plt.tight_layout()
        save_path = os.path.join(self.dirs['plots'], 'per_class_map.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def create_detection_grid(self, images, detections, class_names, model_names):
        n_images = len(images)
        n_models = len(model_names)
        fig, axes = plt.subplots(n_images, n_models, figsize=(5*n_models, 5*n_images))
        for i in range(n_images):
            for j in range(n_models):
                ax = axes[i, j] if n_images > 1 else axes[j]
                ax.imshow(images[i])
                for det in detections[j][i]:
                    bbox = det['bbox']
                    label = class_names[det['class_id']]
                    conf = det['confidence']
                    rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
                                             linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(rect)
                    ax.text(bbox[0], bbox[1]-5, f'{label}: {conf:.2f}',
                            color='white', bbox=dict(facecolor='red', alpha=0.5))
                if i == 0:
                    ax.set_title(model_names[j])
                ax.axis('off')
        plt.tight_layout()
        save_path = os.path.join(self.dirs['detections'], 'detection_grid.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def save_markdown_report(self, metrics_df, additional_notes=None):
        report_path = os.path.join(self.dirs['metrics'], 'complete_report.md')
        with open(report_path, 'w') as f:
            f.write("# Model Comparison Report\n\n")
            f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write("## Summary Metrics\n")
            f.write(metrics_df.to_markdown())
            f.write("\n\n")
            f.write("## Visualization Directory Structure\n")
            for dir_name, dir_path in self.dirs.items():
                f.write(f"- {dir_name}: {dir_path}\n")
            f.write("\n")
            if additional_notes:
                f.write("## Additional Notes\n")
                f.write(additional_notes)
                f.write("\n")
            f.write("\n## Plots Generated\n")
            f.write("1. Metrics Comparison (Bar Plot)\n")
            f.write("2. Confusion Matrices\n")
            f.write("3. Precision-Recall Curves\n")
            f.write("4. Per-Class mAP Comparison\n")
            f.write("5. Detection Examples Grid\n")

    def generate_all_visualizations(self, results_dict, class_names):
        # 1. Overall metrics comparison
        metrics_df = self.plot_metrics_comparison(results_dict['summary_metrics'])
        # 2. Confusion matrices
        for model_name in results_dict['confusion_matrices']:
            cm_data = results_dict['confusion_matrices'][model_name]
            self.plot_confusion_matrix(
                cm_data['true'],
                cm_data['pred'],
                class_names,
                model_name
            )
        # 3. Precision-recall curves
        self.plot_precision_recall_curves(
            results_dict['precision'],
            results_dict['recall'],
            list(results_dict['summary_metrics'].keys())
        )
        # 4. Per-class mAP
        if 'per_class_map' in results_dict:
            self.plot_per_class_map(
                results_dict['per_class_map'],
                class_names,
                list(results_dict['summary_metrics'].keys())
            )
        # 5. Detection grid
        if 'example_images' in results_dict and 'example_detections' in results_dict:
            self.create_detection_grid(
                results_dict['example_images'],
                results_dict['example_detections'],
                class_names,
                list(results_dict['summary_metrics'].keys())
            )
        # 6. Markdown report
        self.save_markdown_report(
            metrics_df,
            additional_notes=results_dict.get('notes', None)
        )
        logger.info(f"All visualizations saved in {self.results_dir}")
        return self.results_dir

In [9]:
class ModelEvaluator:
    def __init__(self, base_dir, data_yaml, dataset_dir):
        self.base_dir = base_dir
        self.data_yaml = data_yaml
        self.dataset_dir = dataset_dir
        self.visualizer = ResultsVisualizer(base_dir)

        # Load class names from yaml
        with open(data_yaml, 'r') as f:
            self.data_config = yaml.safe_load(f)
        self.class_names = self.data_config['names']

        logger.info(f"Initialized evaluator with {len(self.class_names)} classes")

    def train_and_evaluate_baseline(self):
        """Train and evaluate baseline YOLOv8 model"""
        logger.info("Starting baseline model training...")

        model = YOLO('yolov8n.pt')
        results = model.train(
            data=self.data_yaml,
            epochs=1,
            imgsz=640,
            project=self.base_dir,
            name='baseline_model'
        )

        # Evaluate
        val_results = model.val(data=self.data_yaml)
        return model, val_results

    def train_and_evaluate_enhanced(self):
        """Train and evaluate enhanced model (larger size + TTA)"""
        logger.info("Starting enhanced model training...")

        model = YOLO('yolov8n.pt')
        results = model.train(
            data=self.data_yaml,
            epochs=1,
            imgsz=1024,
            project=self.base_dir,
            name='enhanced_model'
        )

        # Evaluate with TTA
        val_results = model.val(
            data=self.data_yaml,
            imgsz=1024,
            augment=True
        )
        return model, val_results

    def evaluate_with_sahi(self, model_path):
        """Evaluate using SAHI with better error handling"""
        try:
            from sahi import AutoDetectionModel
            from sahi.predict import get_sliced_prediction

            logger.info(f"Loading model from {model_path}")
            if not os.path.exists(model_path):
                raise FileNotFoundError(f"Model file not found: {model_path}")

            detection_model = AutoDetectionModel.from_pretrained(
                model_type='ultralytics',
                model_path=model_path,
                confidence_threshold=0.3,
                device='cuda'
            )

            test_images_dir = os.path.join(self.dataset_dir, 'test/images')
            if not os.path.exists(test_images_dir):
                raise FileNotFoundError(f"Test images dir not found: {test_images_dir}")

            results = []
            for image_name in os.listdir(test_images_dir):
                if image_name.endswith(('.jpg', '.png')):
                    image_path = os.path.join(test_images_dir, image_name)
                    try:
                        result = get_sliced_prediction(
                            image=image_path,
                            detection_model=detection_model,
                            slice_height=512,
                            slice_width=512,
                            overlap_height_ratio=0.2,
                            overlap_width_ratio=0.2
                        )
                        results.append(result)
                        logger.debug(f"Processed {image_name} successfully")
                    except Exception as e:
                        logger.warning(f"Failed to process {image_name}: {str(e)}")
                        continue

            return results

        except Exception as e:
            logger.error(f"SAHI evaluation failed: {str(e)}")
            return []

    def run_complete_evaluation(self):
        """Run complete evaluation pipeline"""
        try:
            # 1. Baseline evaluation
            baseline_model, baseline_results = self.train_and_evaluate_baseline()

            # 2. Enhanced evaluation
            enhanced_model, enhanced_results = self.train_and_evaluate_enhanced()

            # 3. SAHI evaluation
            sahi_results = self.evaluate_with_sahi(
                f'{self.base_dir}/enhanced_model/weights/best.pt'
            )

            # Function for safe metric extraction
            def safe_get_metrics(results):
                """Safe metric extraction with fallbacks"""
                if not hasattr(results, 'box'):
                    logger.error("Validation results missing 'box' attribute")
                    return {
                        'mAP50': 0,
                        'mAP50-95': 0,
                        'recall': 0,
                        'precision': 0
                    }

                box = results.box
                return {
                    'mAP50': getattr(box, 'map50', 0),
                    'mAP50-95': getattr(box, 'map', 0),
                    'recall': getattr(box, 'r', 0),
                    'precision': getattr(box, 'p', 0)
                }

            # 4. Collect all results
            baseline_metrics = safe_get_metrics(baseline_results)
            enhanced_metrics = safe_get_metrics(enhanced_results)

            results_dict = {
                'summary_metrics': {
                    'Baseline': {
                        'mAP50': baseline_metrics['mAP50'],
                        'mAP50-95': baseline_metrics['mAP50-95'],
                        'recall': baseline_metrics['recall']
                    },
                    'Enhanced+TTA': {
                        'mAP50': enhanced_metrics['mAP50'],
                        'mAP50-95': enhanced_metrics['mAP50-95'],
                        'recall': enhanced_metrics['recall']
                    }
                },
                # Dummy confusion matrices
                'confusion_matrices': {
                    'Baseline': {
                        'true': [0]*len(self.class_names),
                        'pred': [0]*len(self.class_names)
                    },
                    'Enhanced+TTA': {
                        'true': [0]*len(self.class_names),
                        'pred': [0]*len(self.class_names)
                    }
                }
            }

            # 5. Generate visualizations
            self.visualizer.generate_all_visualizations(
                results_dict,
                self.class_names
            )

            logger.info("Evaluation completed successfully!")
            return results_dict

        except Exception as e:
            logger.error(f"Error during evaluation: {str(e)}", exc_info=True)
            raise

    def test_metrics_extraction(self):
        """Test metrics extraction works with current YOLO version"""
        from unittest.mock import MagicMock

        # Create mock results object
        mock_results = MagicMock()
        mock_results.box = MagicMock()
        mock_results.box.map50 = 0.5
        mock_results.box.map = 0.4
        mock_results.box.r = 0.3
        mock_results.box.p = 0.6
        mock_results.confusion_matrix = None

        # Since safe_get_metrics is inside run_complete_evaluation, redefining it here
        def safe_get_metrics(results):
            if not hasattr(results, 'box'):
                return {
                    'mAP50': 0,
                    'mAP50-95': 0,
                    'recall': 0,
                    'precision': 0
                }

            box = results.box
            return {
                'mAP50': getattr(box, 'map50', 0),
                'mAP50-95': getattr(box, 'map', 0),
                'recall': getattr(box, 'r', 0),
                'precision': getattr(box, 'p', 0)
            }

        metrics = safe_get_metrics(mock_results)
        assert metrics['mAP50'] == 0.5
        assert metrics['recall'] == 0.3
        logger.info("✅ Metrics extraction test passed!")

        # Test missing attribute handling
        mock_results.box = None
        metrics = safe_get_metrics(mock_results)
        assert metrics['mAP50'] == 0
        logger.info("✅ Error handling test passed!")

In [10]:
class ModelEvaluator:
    def __init__(self, base_dir, data_yaml, dataset_dir):
        self.base_dir = base_dir
        self.data_yaml = data_yaml
        self.dataset_dir = dataset_dir
        self.visualizer = ResultsVisualizer(base_dir)

        # Load class names from yaml
        with open(data_yaml, 'r') as f:
            self.data_config = yaml.safe_load(f)
        self.class_names = self.data_config['names']

        logger.info(f"Initialized evaluator with {len(self.class_names)} classes")

    def train_and_evaluate_baseline(self):
        """Train and evaluate baseline YOLOv8 model"""
        logger.info("Starting baseline model training...")

        model = YOLO('yolov8n.pt')
        results = model.train(
            data=self.data_yaml,
            epochs=500,  # change to 2 for quick testing, increase as needed
            imgsz=640,
            project=self.base_dir,
            name='baseline_model'
        )

        # Evaluate
        val_results = model.val(data=self.data_yaml)
        return model, val_results

    def train_and_evaluate_enhanced(self):
        """Train and evaluate enhanced model (larger size + TTA)"""
        logger.info("Starting enhanced model training...")

        model = YOLO('yolov8n.pt')
        results = model.train(
            data=self.data_yaml,
            epochs=500,  # change to 2 for quick testing, increase as needed
            imgsz=1024,
            project=self.base_dir,
            name='enhanced_model'
        )

        # Evaluate with TTA
        val_results = model.val(
            data=self.data_yaml,
            imgsz=1024,
            augment=True
        )
        return model, val_results

    def evaluate_with_sahi(self, model_path):
        """Evaluate using SAHI with better error handling"""
        try:
            from sahi import AutoDetectionModel
            from sahi.predict import get_sliced_prediction

            logger.info(f"Loading model from {model_path}")
            if not os.path.exists(model_path):
                raise FileNotFoundError(f"Model file not found: {model_path}")

            detection_model = AutoDetectionModel.from_pretrained(
                model_type='ultralytics',
                model_path=model_path,
                confidence_threshold=0.3,
                device='cuda'
            )

            test_images_dir = os.path.join(self.dataset_dir, 'test/images')
            if not os.path.exists(test_images_dir):
                raise FileNotFoundError(f"Test images dir not found: {test_images_dir}")

            results = []
            for image_name in os.listdir(test_images_dir):
                if image_name.endswith(('.jpg', '.png')):
                    image_path = os.path.join(test_images_dir, image_name)
                    try:
                        result = get_sliced_prediction(
                            image=image_path,
                            detection_model=detection_model,
                            slice_height=512,
                            slice_width=512,
                            overlap_height_ratio=0.2,
                            overlap_width_ratio=0.2
                        )
                        results.append(result)
                        logger.debug(f"Processed {image_name} successfully")
                    except Exception as e:
                        logger.warning(f"Failed to process {image_name}: {str(e)}")
                        continue

            return results

        except Exception as e:
            logger.error(f"SAHI evaluation failed: {str(e)}")
            return []

    def run_complete_evaluation(self):
        """Run complete evaluation pipeline"""
        try:
            # 1. Baseline evaluation
            baseline_model, baseline_results = self.train_and_evaluate_baseline()

            # 2. Enhanced evaluation
            enhanced_model, enhanced_results = self.train_and_evaluate_enhanced()

            # 3. SAHI evaluation
            sahi_results = self.evaluate_with_sahi(
                f'{self.base_dir}/enhanced_model/weights/best.pt'
            )

            # Function for safe metric extraction
            def safe_get_metrics(results):
                """Safe metric extraction with fallbacks"""
                if not hasattr(results, 'box'):
                    logger.error("Validation results missing 'box' attribute")
                    return {
                        'mAP50': 0,
                        'mAP50-95': 0,
                        'recall': 0,
                        'precision': 0
                    }

                box = results.box
                return {
                    'mAP50': getattr(box, 'map50', 0),
                    'mAP50-95': getattr(box, 'map', 0),
                    'recall': getattr(box, 'r', 0),
                    'precision': getattr(box, 'p', 0)
                }

            # 4. Collect all results
            baseline_metrics = safe_get_metrics(baseline_results)
            enhanced_metrics = safe_get_metrics(enhanced_results)

            # Dummy per-class precision and recall, for each model, for plotting (set to zeros for now)
            precision_dummy = [
                [0]*len(self.class_names),
                [0]*len(self.class_names)
            ]
            recall_dummy = [
                [0]*len(self.class_names),
                [0]*len(self.class_names)
            ]

            results_dict = {
                'summary_metrics': {
                    'Baseline': {
                        'mAP50': baseline_metrics['mAP50'],
                        'mAP50-95': baseline_metrics['mAP50-95'],
                        'recall': baseline_metrics['recall']
                    },
                    'Enhanced+TTA': {
                        'mAP50': enhanced_metrics['mAP50'],
                        'mAP50-95': enhanced_metrics['mAP50-95'],
                        'recall': enhanced_metrics['recall']
                    }
                },
                # Dummy confusion matrices
                'confusion_matrices': {
                    'Baseline': {
                        'true': [0]*len(self.class_names),
                        'pred': [0]*len(self.class_names)
                    },
                    'Enhanced+TTA': {
                        'true': [0]*len(self.class_names),
                        'pred': [0]*len(self.class_names)
                    }
                },
                # Add dummy precision and recall for plotting
                'precision': precision_dummy,
                'recall': recall_dummy
            }

            # 5. Generate visualizations
            self.visualizer.generate_all_visualizations(
                results_dict,
                self.class_names
            )

            logger.info("Evaluation completed successfully!")
            return results_dict

        except Exception as e:
            logger.error(f"Error during evaluation: {str(e)}", exc_info=True)
            raise

    def test_metrics_extraction(self):
        """Test metrics extraction works with current YOLO version"""
        from unittest.mock import MagicMock

        # Create mock results object
        mock_results = MagicMock()
        mock_results.box = MagicMock()
        mock_results.box.map50 = 0.5
        mock_results.box.map = 0.4
        mock_results.box.r = 0.3
        mock_results.box.p = 0.6
        mock_results.confusion_matrix = None

        # Since safe_get_metrics is inside run_complete_evaluation, redefining it here
        def safe_get_metrics(results):
            if not hasattr(results, 'box'):
                return {
                    'mAP50': 0,
                    'mAP50-95': 0,
                    'recall': 0,
                    'precision': 0
                }

            box = results.box
            return {
                'mAP50': getattr(box, 'map50', 0),
                'mAP50-95': getattr(box, 'map', 0),
                'recall': getattr(box, 'r', 0),
                'precision': getattr(box, 'p', 0)
            }

        metrics = safe_get_metrics(mock_results)
        assert metrics['mAP50'] == 0.5
        assert metrics['recall'] == 0.3
        logger.info("✅ Metrics extraction test passed!")

        # Test missing attribute handling
        mock_results.box = None
        metrics = safe_get_metrics(mock_results)
        assert metrics['mAP50'] == 0
        logger.info("✅ Error handling test passed!")

In [11]:
# 5. Main Pipeline

# Initialize evaluator
evaluator = ModelEvaluator(BASE_DIR, DATA_YAML, DATASET_DIR)
evaluator.test_metrics_extraction()

# Run evaluation
results = evaluator.run_complete_evaluation()

logger.info(f"Results saved in: {evaluator.visualizer.results_dir}")

2025-06-08 07:58:40 - INFO - Created results directory at /content/drive/MyDrive/new scope model/comparison_results_20250608_075840
INFO:YOLOComparison:Created results directory at /content/drive/MyDrive/new scope model/comparison_results_20250608_075840
2025-06-08 07:58:40 - INFO - Initialized evaluator with 8 classes
INFO:YOLOComparison:Initialized evaluator with 8 classes
2025-06-08 07:58:40 - INFO - ✅ Metrics extraction test passed!
INFO:YOLOComparison:✅ Metrics extraction test passed!
2025-06-08 07:58:40 - INFO - ✅ Error handling test passed!
INFO:YOLOComparison:✅ Error handling test passed!
2025-06-08 07:58:40 - INFO - Starting baseline model training...
INFO:YOLOComparison:Starting baseline model training...


100%|██████████| 6.25M/6.25M [00:00<00:00, 78.8MB/s]


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/new scope model/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline_model33, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

100%|██████████| 755k/755k [00:00<00:00, 13.3MB/s]

Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        354       2095      0.914      0.863      0.929      0.749
                   Car        194        852      0.966      0.938      0.976       0.85
            Pedestrian        354        881      0.952      0.789      0.914      0.583
                   Van         86        107      0.968      0.963      0.976      0.863
               Cyclist         79        127      0.938      0.898      0.947      0.734
        Person_sitting         16         48      0.835      0.708      0.823      0.616
                  Misc         16         22       0.91      0.773      0.875      0.707
                 Truck         27         29      0.834      0.866      0.944      0.821
                  Tram         10         29      0.913      0.966      0.972      0.817
Speed: 0.6ms preprocess, 4.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/new scope model/enhanced_model252


2025-06-08 11:31:38 - INFO - Loading model from /content/drive/MyDrive/new scope model/enhanced_model/weights/best.pt
INFO:YOLOComparison:Loading model from /content/drive/MyDrive/new scope model/enhanced_model/weights/best.pt


Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing prediction on 3 slices.
Performing predictio

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
2025-06-08 11:37:53 - INFO - All visualizations saved in /content/drive/MyDrive/new scope model/comparison_results_20250608_075840
INFO:YOLOComparison:All visualizations saved in /content/drive/MyDrive/new scope model/comparison_results_20250608_075840
2025-06-08 11:37:53 - INFO - Evaluation completed successfully!
INFO:YOLOComparison:Evaluation completed successfully!
2025-06-08 11:37:53 - INFO - Results saved in: /content/drive/MyDrive/new scope model/comparison

<Figure size 1200x800 with 0 Axes>

In [12]:
# 6. Visualize Detections: Baseline vs Enhanced+TTA vs Enhanced+SAHI (no retraining)

import cv2
import glob
import os

# --- Use trained models from previous cells ---
baseline_model = evaluator.train_and_evaluate_baseline()[0]     # Model object, already trained
enhanced_model = evaluator.train_and_evaluate_enhanced()[0]     # Model object, already trained

# --- Use best weights for SAHI ---
enhanced_best_path = os.path.join(BASE_DIR, "enhanced_model/weights/best.pt")
sahi_results = evaluator.evaluate_with_sahi(enhanced_best_path)  # List of SAHI results

# --- Prepare test images ---
test_images_dir = os.path.join(DATASET_DIR, '/content/drive/MyDrive/Samplesmall_dataset/test/images')
image_exts = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
example_image_paths = []
for ext in image_exts:
    example_image_paths.extend(glob.glob(os.path.join(test_images_dir, ext)))
example_image_paths = sorted(example_image_paths)[:2]
example_images = [cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB) for p in example_image_paths]

# --- Helper functions ---
def extract_yolo_detections(model, image_paths, class_names, conf_thres=0.3):
    detections_all = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        results = model(img, conf=conf_thres)
        dets = []
        for r in results:
            if hasattr(r, 'boxes'):
                for box in r.boxes:
                    xywh = box.xywh[0].cpu().numpy()
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    dets.append({
                        'bbox': [xywh[0]-xywh[2]/2, xywh[1]-xywh[3]/2, xywh[2], xywh[3]],
                        'class_id': cls_id,
                        'confidence': conf
                    })
        detections_all.append(dets)
    return detections_all

def extract_sahi_detections(sahi_results, class_names):
    detections_all = []
    for result in sahi_results:
        dets = []
        for det in getattr(result, 'object_prediction_list', []):
            bbox = det.bbox.to_xywh()
            dets.append({
                'bbox': [bbox[0], bbox[1], bbox[2], bbox[3]],
                'class_id': det.category.id,
                'confidence': det.score.value
            })
        detections_all.append(dets)
    return detections_all

# --- Run detections ---
baseline_detections = extract_yolo_detections(baseline_model, example_image_paths, evaluator.class_names)
enhanced_detections = extract_yolo_detections(enhanced_model, example_image_paths, evaluator.class_names)
sahi_detections = extract_sahi_detections(sahi_results, evaluator.class_names)

# --- Prepare for grid plot ---
detections_list = [baseline_detections, enhanced_detections, sahi_detections]
method_names = ['Baseline', 'Enhanced+TTA', 'Enhanced+SAHI']

# --- Plot comparison grid ---
save_path = os.path.join(evaluator.visualizer.dirs['detections'], 'detection_comparison_grid.png')
plot_comparison_grid(
    images=example_images,
    detections_list=detections_list,
    class_names=evaluator.class_names,
    method_names=method_names,
    save_path=save_path
)
logger.info(f"Detection comparison grid saved to: {save_path}")

2025-06-08 12:10:31 - INFO - Starting baseline model training...
INFO:YOLOComparison:Starting baseline model training...


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/new scope model/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline_model34, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

train: Scanning /content/drive/MyDrive/Samplesmall_dataset/train/labels.cache... 1423 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1423/1423 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.7±0.7 ms, read: 153.1±112.2 MB/s, size: 828.8 KB)


...

                   all        354       2095      0.747      0.752       0.78      0.479

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/500       2.3G      1.185     0.7957      1.036        166        640: 100%|██████████| 89/89 [00:08<00:00,  9.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:01<00:00,  7.24it/s]


                   all        354       2095      0.849      0.714      0.802      0.509

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     42/500       2.3G      1.176     0.7839      1.034        229        640:  87%|████████▋ | 77/89 [00:07<00:01, 10.13it/s]


KeyboardInterrupt: 